# 基准测试

## 计时模块

In [1]:
import time

In [10]:
def timer(func, *args):
    start = time.time()
    for i in range(1000):
        func(*args)
    return time.time() - start

In [11]:
timer(pow, 2, 1000)

0.0009963512420654297

上面的计时函数存在的问题：
- 不支持关键字参数
- 硬编码了重复次数
- 额外计入了range的时间开销
- 总使用time.clock,在windows系统意外并非是最好的
- 无法让调用者检测待检测函数确实有效
- 只给出了总时间，这些在一些重负载的机器上可能会被动

In [12]:
import time, sys

In [68]:
def total(reps, func, *pargs, **kargs):
    repslist = list(range(reps))
    start = time.time()
    for i in repslist:
        ret = func(*pargs, **kargs)
    elapsed = time.time() - start
    return (elapsed, ret)

In [69]:
def bestof(reps, func, *pargs, **kargs):
    best = 2 ** 32
    for i in range(reps):
        start = time.time()
        ret = func(*pargs, **kargs)
        elapsed = time.time() - start
        best = elapsed if elapsed < best else best
    return (best, ret)

In [70]:
def bestoftotal(reps1, reps2, func, *pargs, **kargs):
    return bestof(reps1, total, reps2, func, *pargs, **kargs)

In [49]:
bestof(50, total, 1000, str.upper, 'spam')

(0.0, (0.0, 'SPAM'))

In [42]:
total(1000, pow, 2, 1000)

(0.0009975433349609375,
 10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376)

## 列表生成式计时

In [18]:
reps = 10000
repslist = list(range(reps))

In [19]:
def forLoop():
    res = []
    for x in repslist:
        res.append(abs(x))
    return res

In [20]:
def listComp():
    return [abs(x) for x in repslist]

In [21]:
def mapCall():
    return list(map(abs, repslist))

In [22]:
def genExpr():
    return list(abs(x) for x in repslist)

In [23]:
def genFunc():
    def gen():
        for x in repslist:
            yield abs(x)
    return list(gen())

In [24]:
print(sys.version)

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [71]:
for test in (forLoop, listComp, mapCall, genExpr, genFunc):
    (bestoftime, (totaltime, result)) = bestoftotal(5, 1000, test)
    print('%-9s: %.5f => [%s...%s]' % (test.__name__, bestoftime, result[0], result[-1]))

forLoop  : 0.78989 => [0...9999]
listComp : 0.46276 => [0...9999]
mapCall  : 0.20745 => [0...9999]
genExpr  : 0.68086 => [0...9999]
genFunc  : 0.70720 => [0...9999]


## 用timeit 为迭代和各种python计时

In [72]:
import timeit

In [73]:
# 重复5次，每次执行100遍stmt语句
min(timeit.repeat(stmt="[x ** 2 for x in range(1000)]", number=1000, repeat=5))

0.29871490000004997